In [2]:
import re
import pandas as pd

In [3]:

def preprocessing_dataset(dataset):
  """ 처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
  subject_entity = []
  object_entity = []
  subject_type= []
  object_type= []
  subject_start= []
  object_start= []
  subject_end= []
  object_end= []
  

  for i,j in zip(dataset['subject_entity'], dataset['object_entity']):
    # i = i[1:-1].split(',')[0].split(':')[1]
    sub_entity = eval(i)
    ob_entity = eval(j)
    sub_word= sub_entity['word']
    sub_type = sub_entity['type']
    # j = j[1:-1].split(',')[0].split(':')[1]
    ob_word = ob_entity['word']
    ob_type = ob_entity['type']

    ob_start = ob_entity['start_idx']
    sub_start = sub_entity['start_idx']

    ob_end = ob_entity['end_idx']
    sub_end = sub_entity['end_idx']


    subject_entity.append(sub_word)
    object_entity.append(ob_word)
    subject_type.append(sub_type)
    object_type.append(ob_type)
    subject_start.append(sub_start)
    object_start.append(ob_start)
    subject_end.append(sub_end)
    object_end.append(ob_end)
  
  out_dataset = pd.DataFrame({'id':dataset['id'], 'sentence':dataset['sentence'],'subject_entity':subject_entity,'object_entity':object_entity,'label':dataset['label'],
                              'subject_type' : subject_type, 'object_type': object_type,'sub_start' : subject_start,'ob_start':object_start,'sub_end' :subject_end ,'ob_end': object_end})
  out_dataset.id = out_dataset.index
  return out_dataset


  
def load_data(dataset_dir):
  """ csv 파일을 경로에 맡게 불러 옵니다. """
  pd_dataset = pd.read_csv(dataset_dir)
  dataset = preprocessing_dataset(pd_dataset)
  
  return dataset

### 1. 기본 베이스 모델

In [4]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + ' [SEP] ' + e02
    concat_entity.append(temp)
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)

['Runch Randa [SEP] 방탄소년단', '위키백과 [SEP] 위키미디어 재단', '임재범 [SEP] 송남영', '산청군 [SEP] 이재근', '유신정우회 [SEP] 한국국민당']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 2. 단어를 토큰으로 바꾸기


In [5]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = e01 + ' [SEP] ' + e02
    concat_entity.append(temp)
  
  dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
                                                                  dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['Runch Randa [SEP] 방탄소년단', '위키백과 [SEP] 위키미디어 재단', '임재범 [SEP] 송남영', '산청군 [SEP] 이재근', '유신정우회 [SEP] 한국국민당']
0          <PER> 멤버로 데뷔하기 전에 그는 《<PER>》라는 이름인 언더 래퍼였다.
1    <ORG>를 운영하고 있는 <ORG>은 매년 세계적인 컨퍼런스인 위키마니아 행사를 ...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, <PE...
3    경남 <ORG> <ORG>청에서 열린 협약식에는 최재호 무학그룹 회장과 <PER> ...
4    1980년 구 민주공화당 인사들과 <ORG> 인사들이 <ORG>을 창당하였고, 19...
Name: sentence, dtype: object


### 3. 인덱스 스타트 번호 넣기

In [6]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, s01, s02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['sub_start'], dataset['ob_start']):
    temp = ''
    temp = e01 + ' [SEP] '+ f'{s01}' + ' [SEP] ' + e02+ ' [SEP] '+ f'{s02}'
    concat_entity.append(temp)
  
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #                                                                 dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['Runch Randa [SEP] 22 [SEP] 방탄소년단 [SEP] 0', '위키백과 [SEP] 0 [SEP] 위키미디어 재단 [SEP] 14', '임재범 [SEP] 43 [SEP] 송남영 [SEP] 52', '산청군 [SEP] 7 [SEP] 이재근 [SEP] 36', '유신정우회 [SEP] 19 [SEP] 한국국민당 [SEP] 30']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 4. 스타트 번호, 끝번호 둘다 넣기

In [7]:

def change_word(sentence, sub_word, sub_type , ob_word, ob_type):
  sentence = re.sub(rf'{sub_word}',f'<{sub_type}>',sentence)
  sentence = re.sub(rf'{ob_word}',f'<{ob_type}>',sentence)

  return sentence

def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02, s01, s02, d01, d02 in zip(dataset['subject_entity'], dataset['object_entity'], dataset['sub_start'], dataset['ob_start'], dataset['sub_end'], dataset['ob_end']):
    temp = ''
    temp = e01 + ' [SEP] '+ f'{s01} {d01}' + ' [SEP] ' + e02+ ' [SEP] '+ f'{s02} {d02}'
    concat_entity.append(temp)
  
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #                                                                 dataset.subject_type.loc[x],dataset.object_entity.loc[x], dataset.object_type.loc[x]))
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
                                  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)                                

['Runch Randa [SEP] 22 32 [SEP] 방탄소년단 [SEP] 0 4', '위키백과 [SEP] 0 3 [SEP] 위키미디어 재단 [SEP] 14 21', '임재범 [SEP] 43 45 [SEP] 송남영 [SEP] 52 54', '산청군 [SEP] 7 9 [SEP] 이재근 [SEP] 36 38', '유신정우회 [SEP] 19 23 [SEP] 한국국민당 [SEP] 30 34']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object


### 5. object이랑 subject 각각 표시

In [8]:
def tokenized_dataset(dataset):
  """ tokenizer에 따라 sentence를 tokenizing 합니다."""
  concat_entity = []
  for e01, e02 in zip(dataset['subject_entity'], dataset['object_entity']):
    temp = ''
    temp = '<ob> ' + e01 + ' [SEP] ' + '<sub> ' + e02
    concat_entity.append(temp)
  
  print(concat_entity[:5])
  print(dataset['sentence'].head())
  # dataset['sentence'] = dataset['id'].apply(lambda x: change_word(dataset.sentence.loc[x],dataset.subject_entity.loc[x],
  #  
train_dataset = load_data("../data/dataset/train/train_totalX3.csv")
tokenized_train = tokenized_dataset(train_dataset)

['<ob> Runch Randa [SEP] <sub> 방탄소년단', '<ob> 위키백과 [SEP] <sub> 위키미디어 재단', '<ob> 임재범 [SEP] <sub> 송남영', '<ob> 산청군 [SEP] <sub> 이재근', '<ob> 유신정우회 [SEP] <sub> 한국국민당']
0    방탄소년단 멤버로 데뷔하기 전에 그는 《Runch Randa》라는 이름인 언더 래퍼였다.
1    위키백과를 운영하고 있는 위키미디어 재단은 매년 세계적인 컨퍼런스인 위키마니아 행사...
2    그는 이화여대 미대 출신의 한국 여성 정재은씨와 결혼해 두 딸을 두고 있고, 임재범...
3    경남 산청군 산청군청에서 열린 협약식에는 최재호 무학그룹 회장과 이재근 산청군수 외...
4    1980년 구 민주공화당 인사들과 유신정우회 인사들이 한국국민당을 창당하였고, 19...
Name: sentence, dtype: object
